In [40]:
import pandas as pd

In [41]:
sdf = pd.read_csv('coffee.csv', sep=';', index_col=0)

In [42]:
sdf.drop(labels = [68153], inplace = True)

In [43]:
sdf['Открыт'] = pd.to_datetime(sdf['Открыт'], dayfirst=True)

In [44]:
sdf['Сумма по чеку, руб.'] = sdf['Сумма по чеку, руб.'].apply(lambda x: x.replace(',', ''))

In [45]:
sdf['Сумма по чеку, руб.'] = pd.to_numeric(sdf['Сумма по чеку, руб.'])

In [46]:
sdf.drop(columns={'Номер чека', 'Закрыт', 'Возврат', 'Стоимость доставки, руб.', 'Сумма по чеку без доставки, руб.',
                 'Фискальный регистратор', 'Кассир', 'Официант'}, inplace=True)

In [47]:
sdf['Открыт_дата'] = sdf['Открыт'].apply(lambda x: x.date())

In [48]:
sdf['Открыт_время'] = sdf['Открыт'].apply(lambda x: x.time())

In [49]:
sdf['День недели'] = sdf['Открыт'].dt.dayofweek

In [50]:
sdf['Месяц'] = sdf['Открыт'].dt.month

In [51]:
sdf['Год'] = sdf['Открыт'].dt.year

In [53]:
sdf['Неделя'] = sdf['Открыт_дата'].apply(lambda d: (d.day-1) // 7 + 1)

In [54]:
sdf['ЗП'] = sdf['Неделя'].apply(lambda x: 1 if x in (3, 5) else 0 )

In [55]:
def period(x):
    if x in (12,1,2):
        return 0
    elif x in (3,4,5):
        return 1
    elif x in (6,7,8):
        return 2
    else:
        return 3

In [56]:
sdf['Время года'] = sdf['Месяц'].apply(lambda x: period(x))

In [57]:
sdf = sdf[sdf['Сумма по чеку, руб.']!=0]

In [58]:
sdf = sdf.sort_values('Открыт', ascending=False, ignore_index=True)

In [59]:
sdf.to_csv('raw_data.csv')

In [60]:
sdf_agg = sdf.groupby(['Открыт_дата', 'День недели','Месяц','Год','Неделя','ЗП','Время года'],
                      as_index=False)[['Сумма по чеку, руб.']].agg('sum')

In [61]:
sdf_agg.to_csv('data_agg.csv')